# Amazon Rekognition

**Documentación del api de Rekognition:**
    http://boto3.readthedocs.io/en/latest/reference/services/rekognition.html#client

In [ ]:
import boto3
import subprocess
import uuid
import random
import base64
import os

from IPython.display import Image, display, Markdown, HTML
from PIL import Image as PILimage, ImageDraw


In [ ]:
region = boto3.Session().region_name
rekognition = boto3.client('rekognition', region)

### Funciones de utilería para los laboratorios

In [ ]:
def load_image(file_name):
    with open(file_name, "rb") as imageFile:
      f = imageFile.read()
      return bytearray(f)

In [ ]:
def rand():
    return str(uuid.uuid4())[:8]

In [ ]:
def random_color():
    return (random.randint(1,255),random.randint(1,255),random.randint(1,255))

In [ ]:
def hex_color(color):
    return '#%02x%02x%02x' % color

In [ ]:
def download_image(url, display_image=False):
    file_name = "/tmp/img{0}.{1}".format(rand(),url.split(".")[-1])   
    subprocess.call(["wget","-O",file_name,url])
    if display_image==True:
        display(Image(file_name, width=400))
    return file_name

In [ ]:
def download_s3_image(bucket, key, display_image=False):
    file_name = "/tmp/img{0}.{1}".format(rand(),key.split(".")[-1])
    s3 = boto3.client('s3', region)
    s3.download_file(bucket, key, file_name)
    if display_image==True:
        display(Image(file_name, width=400))
    return file_name

In [ ]:
def bounding_box(bbox, size):
    if len(bbox) != 4:
        return None
    return [ 
        bbox['Left'] * size[0], bbox['Top'] * size[1],
        (bbox['Left'] * size[0]) + bbox['Width'] * size[0], 
        (bbox['Top'] * size[1]) + bbox['Height'] * size[1]
    ]

In [ ]:
def draw_bounding_box(file_name, bbox_response, fill_color='yellow', display_image=False):    
    img = PILimage.open(file_name)
    bbox = bounding_box(bbox_response,img.size)

    draw = ImageDraw.Draw(img)
    draw.line([(bbox[0], bbox[1]), (bbox[2], bbox[1])], fill=fill_color, width=10)
    draw.line([(bbox[2], bbox[1]), (bbox[2], bbox[3])], fill=fill_color, width=10)
    draw.line([(bbox[2], bbox[3]), (bbox[0], bbox[3])], fill=fill_color, width=10)
    draw.line([(bbox[0], bbox[1]), (bbox[0], bbox[3])], fill=fill_color, width=10)
    
    del draw

    extension = os.path.basename(file_name).split(".")[1]
    new_file = "{}/{}.{}".format(os.path.dirname(file_name), rand(), extension)    
    
    if extension.lower()=="png":
        img_format = "PNG"
    else:
        img_format = "JPEG"
        
    img.save(new_file,img_format)
    
    if display_image == True:
        display(Image(new_file, width=200))
    
    return new_file

In [ ]:
def image2base64(filename):
    with open(filename, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
    return encoded_string.decode("utf-8")

In [ ]:
def html_image(file_name, width=200):
    extension = file_name.split('.')[1]
    if extension.lower() == 'png':
        mimetype = "image/png"
    else:
        mimetype = "image/jpeg"
        
    return '<img src="data:%s;base64,%s" width="%spx">' % (mimetype,image2base64(file_name),width)

In [ ]:
def table(columns):
    html_begin = "<div style=margin-top:20px>"
    html_end = "</div>"
    html = html_begin
    
    for column in columns:
        style = 'float:left;margin-left:20px;'
        width = 200
        
        if column['type'] == 'img' 'style' in column :
            style = column['style']
                        
        row = '<div style=%s>' % (style)
        
        if 'header' in column:
            header = '<div style=text-align:center;font-weight:bold;>%s</div>' % (column['header'])
            row = ''.join([row,header])
                
        if column['type'] == 'img':
            if 'width' in column:
                width = column['width']                            
            row = ''.join([row,html_image(column['file'],width=width)])                        
        else:
            if 'style' in column :
                style = column['style']
            rowdata = '<div style="%s">%s</div>' % (style, column['data'])
            row = ''.join([row,rowdata])
        
        if 'footer' in column:
                footer = '<div>%s</div>' % (column['footer'])
                row = ''.join([row,footer])
                               
        row= ''.join([row,'</div>'])
            
        html = ''.join([html,row])
    
    return ''.join([html,html_end])

### Obtención de etiquetas de una imágen localizada en un bucket de S3

In [ ]:

def detect_labels_S3(bucket, key, max_labels=20, min_confidence=80):
    response = rekognition.detect_labels(
        Image={
            "S3Object": {
                "Bucket": bucket,
                "Name": key,
            }
        },
        MaxLabels=max_labels,
        MinConfidence=min_confidence,
    )
    
    result_data = []
    
    image_file = download_s3_image(bucket, key)
    for label in response['Labels']:        
        color = random_color() if len(label['Instances']) > 0 else (0,0,0)
        for instance in label['Instances']:
            image_file = draw_bounding_box(image_file, instance["BoundingBox"], color)
        result_data.append("<span style='color:{}'><b>{}</b></span> - {}% \n".format(hex_color(color), 
                                                                                     label["Name"], 
                                                                                     label["Confidence"]))        
            
    result_data = ''.join(result_data)

    columns = [
        {
            'type':'img',
            'header':"Source Image",
            'file':image_file,
            'width':400
        },
        {
            'type':'data',
            'header':'Result',
            'data':result_data,
            'style':'white-space:pre-wrap;margin-top:10px'
        }
    ]

    display(HTML(table(columns)))
    

In [ ]:
detect_labels_S3("<bucket>", "<key>")

### Obtención de etiquetas de una imágen localizada en el file system local

In [ ]:
def detect_labels(file_name, max_labels=20, min_confidence=80):
    response = rekognition.detect_labels(
        Image={'Bytes': load_image(file_name)},
        MaxLabels=max_labels,
        MinConfidence=min_confidence,
    )

    result_data = []
    
    for label in response['Labels']:        
        color = random_color() if len(label['Instances']) > 0 else (0,0,0)
        for instance in label['Instances']:
            file_name = draw_bounding_box(file_name, instance["BoundingBox"], color)
        result_data.append("<span style='color:{}'><b>{}</b></span> - {}% \n".format(hex_color(color), 
                                                                                     label["Name"], 
                                                                                     label["Confidence"])) 
            
    result_data = ''.join(result_data)

    columns = [
        {
            'type':'img',
            'header':"Source Image",
            'file':file_name,
            'width':400
        },
        {
            'type':'data',
            'header':'Result',
            'data':result_data,
            'style':'white-space:pre-wrap;margin-top:10px'
        }
    ]

    display(HTML(table(columns)))


In [ ]:
source_image = download_image("<url>")    
detect_labels(source_image, max_labels=20, min_confidence=70)

### Detección de rostros en fotos - rekognition.detect_faces( )

In [ ]:
def detect_faces(url):
    file_name = download_image(url)
    
    response = rekognition.detect_faces(
        Image={'Bytes': load_image(file_name)},
        Attributes=['ALL'],
    )

    for face in response['FaceDetails']:
        emotions_data = []
        for emotion in face["Emotions"]:
            emotions_data.append("<b>{Type}</b> - {Confidence}% \n".format(**emotion))

        emotions_data = ''.join(emotions_data)

        result_data = ["<b>Confidence</b>:  {Confidence}% \n".format(**face)]
        result_data.append("<b>Age range</b>:  {AgeRange[Low]} - {AgeRange[High]} \n".format(**face))
        result_data.append("<b>Smile</b>:  {Smile[Value]} \n".format(**face) )
        result_data.append("<b>Eyeglasses</b>:  {Eyeglasses[Value]} \n".format(**face))
        result_data.append("<b>Gender</b>:  {Gender[Value]} \n".format(**face))
        result_data.append("<b>Beard</b>:  {Beard[Value]} \n".format(**face))
        result_data.append("<b>Mustache</b>:  {Mustache[Value]} \n".format(**face))
        result_data.append("<b>EyesOpen</b>:  {EyesOpen[Value]} \n".format(**face))
        result_data.append("<b>Mouth open</b>:  {MouthOpen[Value]} \n".format(**face))    
        result_data = ''.join(result_data)

        columns = [
            {
                'type':'img',
                'header':"Source Face",
                'file':draw_bounding_box(file_name,face["BoundingBox"])
            },
            {
                'type':'data',
                'header':'Result',
                'data':result_data,
                'style':'white-space:pre-wrap;margin-top:20px'
            },
            {
                'type':'data',
                'data':emotions_data,
                'style':'white-space:pre-wrap;margin-top:40px'
            }
        ]

        display(HTML(table(columns)))

In [ ]:
detect_faces("<url>")

### Comparación de rostros de dos fotos - rekognition.compare_faces( )

In [ ]:
def compare_faces(source_url, target_url, threshold=80):
    source_image = download_image(source_url)
    target_image = download_image(target_url)
    
    response = rekognition.compare_faces(
        SourceImage={
            'Bytes': load_image(source_image)
        },
        TargetImage={
            'Bytes': load_image(target_image)
        },
        SimilarityThreshold=threshold,
    )

    source_face = response['SourceImageFace']

    for match in response['FaceMatches']:
        columns = [{
            'type':'img',
            'header':"Source Face",
            'file':draw_bounding_box(source_image,source_face["BoundingBox"]),
            'footer':''.join(["Confidence: ",str(round(source_face['Confidence'],4)),"%"])
        }]

        columns.append({
            'type':'img',
            'header':"Target Face",
            'file':draw_bounding_box(target_image,match['Face']['BoundingBox']),
            'footer':''.join(['Confidence: ',str(round(match['Face']['Confidence'],4)),"%"])
        })

        columns.append({
            'type':'data',
            'header':'Result',
            'data':''.join(['Similarity: ',str(match['Similarity']),"%"]),
            'style':'padding-top:60px'
        })

        display(HTML(table(columns)))

In [ ]:
source_url = "<url>"
target_url = "<url>"
compare_faces(source_url, target_url)

### Creación de una colección para indexar fotos - rekognition.create_collection( )

In [ ]:

collection_id = "my-collection-{0}".format(rand())

rekognition.create_collection(CollectionId=collection_id)

### Indexación de fotos - rekognition.index_faces( )

In [ ]:
def index_faces(source_url, collection_id):
    source_image = download_image(source_url)
    image_id = source_image.split("/")[-1]
    
    response = rekognition.index_faces(
        Image={
            'Bytes': load_image(source_image)
        },
        CollectionId=collection_id,
        ExternalImageId=image_id,
        DetectionAttributes=['ALL'],
    )

    for record in response['FaceRecords']:
        face = record['Face']
        details = record['FaceDetail']


        emotions_data = []
        for emotion in details["Emotions"]:
            emotions_data.append("<b>{Type}</b> - {Confidence}% \n".format(**emotion))

        emotions_data.append('\n\n<b>Face id</b>:\n{FaceId}\n<b>External image id</b>:\n{ExternalImageId}'.format(**face))
        emotions_data = ''.join(emotions_data)

        result_data = ["<b>Confidence</b>:  {Confidence}% \n".format(**details)]
        result_data.append("<b>Age range</b>:  {AgeRange[Low]} - {AgeRange[High]} \n".format(**details))
        result_data.append("<b>Smile</b>:  {Smile[Value]} \n".format(**details) )
        result_data.append("<b>Eyeglasses</b>:  {Eyeglasses[Value]} \n".format(**details))
        result_data.append("<b>Gender</b>:  {Gender[Value]} \n".format(**details))
        result_data.append("<b>Beard</b>:  {Beard[Value]} \n".format(**details))
        result_data.append("<b>Mustache</b>:  {Mustache[Value]} \n".format(**details))
        result_data.append("<b>EyesOpen</b>:  {EyesOpen[Value]} \n".format(**details))
        result_data.append("<b>Mouth open</b>:  {MouthOpen[Value]} \n".format(**details))      
        result_data = ''.join(result_data)

        columns = [
            {
                'type':'img',
                'header':'Source Face',
                'file':draw_bounding_box(source_image,face["BoundingBox"])
            },
            {
                'type':'data',
                'header':'Result',
                'data':result_data,
                'style':'white-space:pre-wrap;margin-top:20px'
            },
            {
                'type':'data',
                'data':emotions_data,
                'style':'white-space:pre-wrap;margin-top:40px'
            }
        ]

        display(HTML(table(columns)))

In [ ]:
index_faces("<url>", collection_id)

In [ ]:
index_faces("<url>", collection_id)

In [ ]:
collection_faces = rekognition.list_faces(CollectionId=collection_id)
display(collection_faces)

In [ ]:
display("Number of images indexed: {0}".format(len(collection_faces['Faces'])))

### Búsqueda de rostros similares a una foto - rekognition.search_faces_by_image( )

In [ ]:
def search_faces_by_image(source_url, collection_id, threshold=80):
    source_image = download_image(source_url)
    
    response = rekognition.search_faces_by_image(
        Image={
            'Bytes': load_image(source_image)
        },
        CollectionId=collection_id,
        FaceMatchThreshold=threshold,
    )
   
    for record in response['FaceMatches']:
        face = record['Face']
        
        result_data = ["<b>Similarity</b>:  {Similarity}% \n".format(**record)]
        result_data.append("<b>Face Id</b>:  {FaceId} \n".format(**face))
        result_data.append("<b>Image Id</b>:  {ImageId} \n".format(**face) )
        result_data.append("<b>External Image Id</b>:  {ExternalImageId} \n".format(**face))
        result_data.append("<b>Confidence</b>:  {Confidence} \n".format(**face))  
        result_data = ''.join(result_data)

        columns = [
            {
                'type':'img',
                'header':'Source Face',
                'file':draw_bounding_box(source_image,response["SearchedFaceBoundingBox"]),
                'footer':'<b>Confidence</b>: {SearchedFaceConfidence}'.format(**response)
            },
            {
                'type':'img',
                'header':'Found Face',
                'file':draw_bounding_box('/tmp/{ExternalImageId}'.format(**face),face["BoundingBox"]),
                'footer':'<b>Confidence</b>: {SearchedFaceConfidence}'.format(**response)
            },
            {
                'type':'data',
                'header':'Result',
                'data':result_data,
                'style':'white-space:pre-wrap;margin-top:20px'
            }
        ]

        display(HTML(table(columns)))


In [ ]:
image_url = "<url>"

display(Markdown("### Busqueda con treshold de 80"))          
search_faces_by_image(image_url,collection_id, threshold=80)

### Reconocimiento de celebridades - rekognition.recognize_celebrities( )

In [ ]:
def recognize_celebrities(image_url):
    file_name = download_image(image_url)
    response = rekognition.recognize_celebrities(
        Image={'Bytes': load_image(file_name)}
    )
    
    for celebrityFace in response['CelebrityFaces']: 
        face = celebrityFace["Face"]
        
        result_data = []
        result_data.append("<b>Match Confidence</b> - {MatchConfidence}% \n".format(**celebrityFace))
        result_data.append("<b>Name</b> - {Name} \n".format(**celebrityFace))
        result_data.append("<b>URLs</b>:\n")        
        
        urls_data = []
        for url in celebrityFace["Urls"]:
            urls_data.append('<a href="http://{0}">{0}</a> \n'.format(url))
        
        urls_data = ''.join(urls_data)
        result_data.append(urls_data)
        
        result_data = ''.join(result_data)

        columns = [
            {
                'type':'img',
                'header':"Source Image",
                'file':draw_bounding_box(file_name,face["BoundingBox"]),
                'footer':'<b>Confidence</b>: {Confidence}'.format(**face),
                'width':400
            },
            {
                'type':'data',
                'header':'Result',
                'data':result_data,
                'style':'white-space:pre-wrap;margin-top:10px'
            }
        ]

        display(HTML(table(columns)))
        

In [ ]:
recognize_celebrities("<url>")

In [ ]:
recognize_celebrities("<url>")

### Identificación de texto en imágenes - rekognition.detect_text( )

In [ ]:
def detect_text(url):
    file_name = download_image(url)
    
    response = rekognition.detect_text(
        Image={'Bytes': load_image(file_name)}
    )
    
    for text in response['TextDetections']: 
        
        result_data = []
        result_data.append("<b>Confidence</b> - {Confidence}% \n".format(**text))
        result_data.append("<b>Detected text</b> - {DetectedText} \n".format(**text))
        result_data.append("<b>Type</b> - {Type} \n".format(**text))        
        result_data.append("<b>Id</b> - {Id} \n".format(**text))    
        if 'ParentId' in text:
            result_data.append("<b>Parent Id</b> - {ParentId} \n".format(**text))        
        
        result_data = ''.join(result_data)

        columns = [
            {
                'type':'img',
                'header':"Source Image",
                'file':draw_bounding_box(file_name,text['Geometry']['BoundingBox']),
                'width':400
            },
            {
                'type':'data',
                'header':'Result',
                'data':result_data,
                'style':'white-space:pre-wrap;margin-top:10px'
            }
        ]

        display(HTML(table(columns)))
        

In [ ]:
detect_text("<url>")

### Limpieza :)

In [ ]:
!rm -f /tmp/*

In [ ]:
response = rekognition.list_collections()
for collectionId in response['CollectionIds']:
    rekognition.delete_collection(CollectionId=collectionId)